In [ ]:
!pip install transformers
!pip install tensorflow-text
import pandas as pd
import seaborn as sns
import tensorflow as tf
from torch import tensor
import tensorflow_hub as hub
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import tensorflow_text as text

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
df = pd.read_csv('/content/drive/MyDrive/Thesis Work 2021-2022/Latest dataset/depression_detection.csv',encoding='Latin-1')
df.head()

,Unnamed: 0,text,class
0,2.0,"Ex Wife Threatening Suicide. Recently, I left ...",depressed
1,3.0,Am I weird I don't get affected by compliments...,non-depressed
2,4.0,Finally 2020 is almost over... So I can never ...,non-depressed
3,8.0,I need help. Just help me. I'm crying so hard.,depressed
4,9.0,"Iâm so lostHello, my name is Adam (16) and I...",depressed


In [17]:
df['label']=df['class'].apply(lambda x: 1 if x=='depressed' else 0)
df.head()

,Unnamed: 0,text,class,label
0,2.0,"Ex Wife Threatening Suicide. Recently, I left ...",depressed,1
1,3.0,Am I weird I don't get affected by compliments...,non-depressed,0
2,4.0,Finally 2020 is almost over... So I can never ...,non-depressed,0
3,8.0,I need help. Just help me. I'm crying so hard.,depressed,1
4,9.0,"Iâm so lostHello, my name is Adam (16) and I...",depressed,1


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'],df['label'],test_size=0.3)
X_train.head(4)

90744     Anyone wanna chat with me? Just send me a chat...
127298    Suicide is starting to seem a realistic option...
228290    i want to die but i dont want to sufferi hate ...
224565    iâve accepted the fact that iâm a lost cau...
Name: text, dtype: object

In [23]:
from transformers import BertTokenizer

In [24]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [25]:
def convert_example_to_feature(review):
  return tokenizer.encode_plus(review,
                add_special_tokens = True, # add [CLS], [SEP]
                max_length = max_length, # max length of the text that can go to BERT
                pad_to_max_length = True, # add [PAD] tokens
                return_attention_mask = True, # add attention mask to not focus on pad tokens
              )

In [26]:
# can be up to 512 for BERT
max_length = 512
batch_size = 6

In [27]:
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

In [29]:
import tensorflow_datasets as tfds

In [32]:
def encode_examples(ds, limit=-1):
  # prepare list, so that we can build up final TensorFlow dataset from slices.
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  label_list = []
  if (limit > 0):
      ds = ds.take(limit)
  for review, label in ds:
    bert_input = convert_example_to_feature(review.decode())
    input_ids_list.append(bert_input['input_ids'])
    token_type_ids_list.append(bert_input['token_type_ids'])
    attention_mask_list.append(bert_input['attention_mask'])
    label_list.append([label])
  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)

In [30]:
ds_train, ds_test = train_test_split(df,test_size=0.3)
ds_train.head(4)

,Unnamed: 0,text,class,label
150939,226486.0,Yo anyone else moving out in like 8 months? I ...,non-depressed,0
125779,189002.0,"Some of my favorite lines from tonight ""Poor b...",non-depressed,0
167589,251457.0,Im like ð¤ close to manifesting spork Yes my...,non-depressed,0
31294,46998.0,i dont know what to doi want to commit suicide...,depressed,1


In [33]:
# train dataset
ds_train_encoded = encode_examples(ds_train).shuffle(10000).batch(batch_size)
# test dataset
ds_test_encoded = encode_examples(ds_test).batch(batch_size)

ValueError: ignored

In [21]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='softmax', name="output")(l)
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [22]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=3, batch_size = 100)


ValueError: ignored

In [12]:
model.fit(X_train, y_train, epochs=2, batch_size = 32)

ValueError: ignored